In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.functions import to_timestamp

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1607211789834_0007,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark=SparkSession.builder.appName('data_cleaning').getOrCreate()

# obtención de datos crudos
raw_df=spark.read.csv('s3://covid19-bd-bucket/data/raw/Casos-COVID_19-Colombia.csv',inferSchema=True,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Lista de campos
raw_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- fecha reporte web: string (nullable = true)
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: string (nullable = true)
 |-- Código DIVIPOLA departamento: integer (nullable = true)
 |-- Nombre departamento: string (nullable = true)
 |-- Código DIVIPOLA municipio: integer (nullable = true)
 |-- Nombre municipio: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Unidad de medida de edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo de contagio: string (nullable = true)
 |-- Ubicación del caso: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- Código ISO del país: integer (nullable = true)
 |-- Nombre del país: string (nullable = true)
 |-- Recuperado: string (nullable = true)
 |-- Fecha de inicio de síntomas: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha de diagnóstico: string (nullable = true)
 |-- Fecha de recuperación: string (nullable = true)
 |-- Tipo de r

# Inicio de pre-procesado de datos
Lista de posibles valores de cada campo. Se omitirán las columnas de fechas ya que se sabe que estos campos solo van a tener fechas o elementos nulos y además hay muchos resultados como para analizarlos de esta manera

In [4]:
raw_df.select("ID de caso").distinct().show()
raw_df.select("Código DIVIPOLA departamento").distinct().show()
raw_df.select("Nombre departamento").distinct().show()
raw_df.select("Código DIVIPOLA municipio").distinct().show()
raw_df.select("Nombre municipio").distinct().show()
raw_df.select("Edad").distinct().show()
raw_df.select("Unidad de medida de edad").distinct().show()
raw_df.select("Sexo").distinct().show()
raw_df.select("Tipo de contagio").distinct().show()
raw_df.select("Ubicación del caso").distinct().show()
raw_df.select("Estado").distinct().show()
raw_df.select("Código ISO del país").distinct().show()
raw_df.select("Nombre del país").distinct().show()
raw_df.select("Recuperado").distinct().show()
raw_df.select("Tipo de recuperación").distinct().show()
raw_df.select("Pertenencia étnica").distinct().show()
raw_df.select("Nombre del grupo étnico").distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|ID de caso|
+----------+
|       148|
|       496|
|       833|
|      1088|
|      1342|
|      1591|
|      2122|
|      2366|
|      3918|
|      4101|
|      4519|
|      6357|
|      7880|
|      8638|
|      9465|
|      9852|
|     10206|
|     10362|
|     10623|
|     10817|
+----------+
only showing top 20 rows

+----------------------------+
|Código DIVIPOLA departamento|
+----------------------------+
|                          81|
|                          76|
|                           8|
|                          11|
|                          99|
|                          66|
|                          85|
|                          44|
|                          91|
|                          94|
|                          54|
|                          19|
|                          23|
|                          70|
|                          95|
|                          47|
|                          52|
|                          20|
|          

In [5]:
# Aquí se va a imprimir el primer valor de fecha reporte web
# para ver como es el formato de fecas que se usan

raw_df.select('fecha reporte web').head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(fecha reporte web='6/3/2020 0:00:00')

## Homogeneización de valores
En esta parte se van a corregir los valores que sean distintos pero que representen el mismo valor
los valores de fechas nulos no se van a tocar y se entenderan como valores desconocidos.

### Cambio de tipos
Hay algunas columnas que tienen datos que se pueden simplificar a un tipo de dato más sencillo debido a que el formato actual no solo consume más memoria y procesamiento sino que además dificulta el posterior análisis del mismo. Otro posible caso es cuándo un tipo de dato se analiza mejor estando en otro formato cómo por ejemplo las fechas que se procesan mejor estando en formato **datetime** en vez de en formato **string**. Las columnas que van a pasar por el cambio de tipo de dato son:
- Recuperado
- fecha reporte web
- Fecha de notificación
- Fecha de inicio de síntomas
- Fecha de muerte
- Fecha de diagnóstico
- Fecha de recuperación

In [6]:
from datetime import datetime

@udf
def recuperadoToBool(recuperado):
    recuperado = recuperado.upper()
    if(recuperado == "RECUPERADO"):
        return True
    elif(recuperado == 'N/A'):
        return None
    else:
        return False
 

raw_df = raw_df.withColumn('fecha reporte web', to_timestamp(raw_df['fecha reporte web'], "d/M/yyyy H:mm:ss"))
raw_df = raw_df.withColumn('Fecha de notificación', to_timestamp(raw_df['Fecha de notificación'], "d/M/yyyy H:mm:ss"))
raw_df = raw_df.withColumn('Fecha de inicio de síntomas', to_timestamp(raw_df['Fecha de inicio de síntomas'], "d/M/yyyy H:mm:ss"))
raw_df = raw_df.withColumn('Fecha de muerte', to_timestamp(raw_df['Fecha de muerte'], "d/M/yyyy H:mm:ss"))
raw_df = raw_df.withColumn('Fecha de diagnóstico', to_timestamp(raw_df['Fecha de diagnóstico'], "d/M/yyyy H:mm:ss"))
raw_df = raw_df.withColumn('Fecha de recuperación', to_timestamp(raw_df['Fecha de recuperación'], "d/M/yyyy H:mm:ss"))

raw_df = raw_df.withColumn('Recuperado', recuperadoToBool(raw_df['Recuperado']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
print(raw_df.select('fecha reporte web').head())
print(raw_df.select('Fecha de notificación').head())
print(raw_df.select('Fecha de inicio de síntomas').head())
print(raw_df.select('Fecha de muerte').head())
print(raw_df.select('Fecha de diagnóstico').head())
print(raw_df.select('Fecha de recuperación').head())

raw_df.select('Recuperado').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(fecha reporte web=datetime.datetime(2020, 3, 6, 0, 0))
Row(Fecha de notificación=datetime.datetime(2020, 3, 2, 0, 0))
Row(Fecha de inicio de síntomas=datetime.datetime(2020, 2, 27, 0, 0))
Row(Fecha de muerte=None)
Row(Fecha de diagnóstico=datetime.datetime(2020, 3, 6, 0, 0))
Row(Fecha de recuperación=datetime.datetime(2020, 3, 13, 0, 0))
+----------+
|Recuperado|
+----------+
|      null|
|      true|
|     false|
+----------+

### mezcla de letras mayúsculas y minúsculas
hay columnas que cuentan con valores distintos pero que representan lo mismo (mezcla de letras maypusculas y minúsculas) por lo que se va a pasar a mover todo a mayúsculas
una opción sería mover todo a números pero esto podría dificultar el análisis posterior
Las columnas que van a pasar por este proceso son:
- Tipo de contagio
- Ubicación del caso
- Estado

In [8]:
@udf
def toUppercase(string):
    return string.upper()

raw_df = raw_df.withColumn('Tipo de contagio',toUppercase(raw_df['Tipo de contagio']))
raw_df = raw_df.withColumn('Ubicación del caso',toUppercase(raw_df['Ubicación del caso']))
raw_df = raw_df.withColumn('Estado',toUppercase(raw_df['Estado']))

raw_df.select('Tipo de contagio').distinct().show()
raw_df.select('Ubicación del caso').distinct().show()
raw_df.select('Estado').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+
|Tipo de contagio|
+----------------+
|     RELACIONADO|
|       IMPORTADO|
|      EN ESTUDIO|
+----------------+

+------------------+
|Ubicación del caso|
+------------------+
|              CASA|
|          HOSPITAL|
|         FALLECIDO|
|      HOSPITAL UCI|
|               N/A|
+------------------+

+---------+
|   Estado|
+---------+
|FALLECIDO|
|    GRAVE|
|     LEVE|
| MODERADO|
|      N/A|
+---------+

## Guardado de datos
Una vez ya se han limpiado los datos, se van a guardar en la ruta **data/curated** del bucket en s3

In [11]:
URI = sc._gateway.jvm.java.net.URI 
Path = sc._gateway.jvm.org.apache.hadoop.fs.Path 
FileSystem = sc._gateway.jvm.org.apache.hadoop.fs.FileSystem 

fs = FileSystem.get(URI("s3n://covid19-bd-bucket"), sc._jsc.hadoopConfiguration()) 
fs.delete(Path("s3n://covid19-bd-bucket/data/curated/Casos-COVID_19-Colombia/")) 
raw_df.write.save("s3://covid19-bd-bucket/data/curated/Casos-COVID_19-Colombia", format='csv', header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…